In [2]:
import numpy as np 
import pandas as pd 
import os



In [22]:
from bs4 import BeautifulSoup


In [21]:
pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1277 sha256=cbcb27c474f76cc3a6fffdd0044e707518035bc8d55b3830306375ea84df68b1
  Stored in directory: c:\users\samar\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [4]:
# The Natural Language Toolkit, or more commonly NLTK, is a suite of libraries and programs for symbolic and 
# statistical natural language processing for English written in the Python programming language.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import re

#TQDM is a progress bar library with good support for nested loops and Jupyter/IPython notebooks.
from tqdm import tqdm

In [10]:
from keras.utils import to_categorical
import random
#from tensorflow import set_random_seed
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
import tensorflow


In [11]:
tensorflow.random.set_seed(123)

In [13]:
pwd

'C:\\Users\\samar\\Sentiment-Analysis-CNN-master'

In [76]:
train=pd.read_csv(r"C:\Users\samar\Sentiment-Analysis-CNN-master\data\d\train.tsv" , sep="\t")
test=pd.read_csv(r"C:\Users\samar\Sentiment-Analysis-CNN-master\data\d\test.tsv" , sep="\t")

test.head()
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [46]:
train['Sentiment'] = train['Sentiment'].replace(1, 0)
train['Sentiment'] = train['Sentiment'].replace(2, 0)
train['Sentiment'] = train['Sentiment'].replace(3, 0)
train['Sentiment'] = train['Sentiment'].replace(4, 1)
train['Sentiment'] = train['Sentiment'].replace(5, 1)

In [47]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,0
1,2,1,A series of escapades demonstrating the adage ...,0
2,3,1,A series,0
3,4,1,A,0
4,5,1,series,0


In [49]:
import string
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

train['Text_Clean'] = train['Phrase'].apply(lambda x: remove_punct(x))


In [55]:
tokens = [word_tokenize(sen) for sen in train.Text_Clean]

In [56]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [57]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [58]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [59]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 

In [60]:
result = [' '.join(sen) for sen in filtered_words] 

In [61]:
train['Text_Final'] = result

In [62]:
train['tokens'] = filtered_words

In [63]:
train[:5]

,PhraseId,SentenceId,Phrase,Sentiment,Text_Clean,Text_Final,tokens
0,1,1,A series of escapades demonstrating the adage ...,0,A series of escapades demonstrating the adage ...,series escapades demonstrating adage good goos...,"[series, escapades, demonstrating, adage, good..."
1,2,1,A series of escapades demonstrating the adage ...,0,A series of escapades demonstrating the adage ...,series escapades demonstrating adage good goose,"[series, escapades, demonstrating, adage, good..."
2,3,1,A series,0,A series,series,[series]
3,4,1,A,0,A,,[]
4,5,1,series,0,series,series,[series]


In [64]:
train = train[['Text_Final', 'tokens', 'Sentiment']]

In [65]:
train[:5]

,Text_Final,tokens,Sentiment
0,series escapades demonstrating adage good goos...,"[series, escapades, demonstrating, adage, good...",0
1,series escapades demonstrating adage good goose,"[series, escapades, demonstrating, adage, good...",0
2,series,[series],0
3,,[],0
4,series,[series],0


In [68]:
X_train,X_val,y_train,y_val=train_test_split(train['Text_Final'],train['Sentiment'],test_size=0.2)

In [69]:
unique_words = set()
len_max = 0

for sent in tqdm(X_train):
    
    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max = len(sent)
        
#length of the list of unique_words gives the no of unique words
print(len(list(unique_words)))
print(len_max)

100%|██████████████████████████████████████████████████████████████████████| 124848/124848 [00:00<00:00, 403091.14it/s]

38
205


In [70]:
tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(X_train))

#texts_to_sequences(texts)

    # Arguments- texts: list of texts to turn to sequences.
    #Return: list of sequences (one per text input).
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(test_sentences)

#padding done to equalize the lengths of all input reviews. LSTM networks needs all inputs to be same length.
#Therefore reviews lesser than max length will be made equal using extra zeros at end. This is padding.

X_train = sequence.pad_sequences(X_train, maxlen=len_max)
X_val = sequence.pad_sequences(X_val, maxlen=len_max)
X_test = sequence.pad_sequences(X_test, maxlen=len_max)

print(X_train.shape,X_val.shape,X_test.shape)

(124848, 205) (31212, 205) (66292, 205)


In [71]:
early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor='val_acc', patience = 2)
callback = [early_stopping]

In [74]:
model=Sequential()
model.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.005),metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 205, 300)          11400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 205, 128)          219648    
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 287,158
Trainable params: 287,158
Non-trainable params: 0
________________________________________________

In [75]:

history=model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=6, batch_size=256, verbose=1, callbacks=callback)

Epoch 1/6


ValueError: in user code:

    C:\Users\samar\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\samar\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\samar\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\samar\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\samar\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\samar\anaconda3\envs\tf-gpu\lib\sit

In [51]:
def clean_sentences(df):
    reviews = []

    for sent in tqdm(df['Phrase']):
        
        #remove html content
        review_text = BeautifulSoup(sent).get_text()
        
        #remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
        #tokenize the sentences
        words = word_tokenize(review_text.lower())
    
        #lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
    
        reviews.append(lemma_words)

    return(reviews)

In [40]:
train_sentences = clean_sentences(train)
test_sentences = clean_sentences(test)
print(len(train_sentences))
print(len(test_sentences))

100%|██████████████████████████████████████████████████████████████████████████| 66292/66292 [00:36<00:00, 1838.50it/s]

156060
66292


In [25]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [41]:
train[:50]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,0
1,2,1,A series of escapades demonstrating the adage ...,0
2,3,1,A series,0
3,4,1,A,0
4,5,1,series,0
5,6,1,of escapades demonstrating the adage that what...,0
6,7,1,of,0
7,8,1,escapades demonstrating the adage that what is...,0
8,9,1,escapades,0
9,10,1,demonstrating the adage that what is good for ...,0


In [90]:
test['Sentiment']=-999
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,-999
1,156062,8545,An intermittently pleasing but mostly routine ...,-999
2,156063,8545,An,-999
3,156064,8545,intermittently pleasing but mostly routine effort,-999
4,156065,8545,intermittently pleasing but mostly routine,-999


In [91]:
df=pd.concat([train,test],ignore_index=True)
print(df.shape)
df.tail()

(222352, 4)


,PhraseId,SentenceId,Phrase,Sentiment
222347,222348,11855,"A long-winded , predictable scenario .",-999
222348,222349,11855,"A long-winded , predictable scenario",-999
222349,222350,11855,"A long-winded ,",-999
222350,222351,11855,A long-winded,-999
222351,222352,11855,predictable scenario,-999


In [92]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [93]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [94]:
df['clean_review']=clean_review(df.Phrase.values)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,A series of escapades demonstrating the adage ...,1,a series of escapade demonstrating the adage t...
1,2,1,A series of escapades demonstrating the adage ...,2,a series of escapade demonstrating the adage t...
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [97]:
tokens1 = [word_tokenize(sen) for sen in df.clean_review]

In [98]:
filtered_words1 = [remove_stop_words(sen) for sen in tokens1] 

In [99]:
result1 = [' '.join(sen) for sen in filtered_words1] 

In [100]:
df['clean_review']=result1

In [101]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,A series of escapades demonstrating the adage ...,1,series escapade demonstrating adage good goose...
1,2,1,A series of escapades demonstrating the adage ...,2,series escapade demonstrating adage good goose
2,3,1,A series,2,series
3,4,1,A,2,
4,5,1,series,2,series


In [102]:
df_train=df[df.Sentiment!=-999]
df_train.shape

(156060, 5)

In [103]:
df_test=df[df.Sentiment==-999]
df_test.drop('Sentiment',axis=1,inplace=True)
print(df_test.shape)
df_test.head()

(66292, 4)


C:\Users\samar\anaconda3\envs\tf-gpu\lib\site-packages\pandas\core\frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,PhraseId,SentenceId,Phrase,clean_review
156060,156061,8545,An intermittently pleasing but mostly routine ...,intermittently pleasing mostly routine effort
156061,156062,8545,An intermittently pleasing but mostly routine ...,intermittently pleasing mostly routine effort
156062,156063,8545,An,
156063,156064,8545,intermittently pleasing but mostly routine effort,intermittently pleasing mostly routine effort
156064,156065,8545,intermittently pleasing but mostly routine,intermittently pleasing mostly routine


In [104]:
train_text=df_train.clean_review.values
test_text=df_test.clean_review.values
target=df_train.Sentiment.values
y=to_categorical(target)
print(train_text.shape,target.shape,y.shape)

(156060,) (156060,) (156060, 5)


In [105]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

(124848,) (124848, 5)
(31212,) (31212, 5)


In [106]:
all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

13604

In [107]:
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

29

In [118]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 5
num_classes=5

In [109]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(test_text)

In [110]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_val.shape,X_test.shape)

(124848, 29) (31212, 29) (66292, 29)


In [111]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         1360400   
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 165       
Total params: 1,415,221
Trainable params: 1,415,221
Non-trainable params: 0
_________________________________________________________________


In [119]:
%%time
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/5
976/976 [==============================] - 65s 67ms/step - loss: 0.7482 - accuracy: 0.6937 - val_loss: 0.8449 - val_accuracy: 0.6614
Epoch 2/5
976/976 [==============================] - 67s 69ms/step - loss: 0.7178 - accuracy: 0.7052 - val_loss: 0.8509 - val_accuracy: 0.6585
Epoch 3/5
976/976 [==============================] - 68s 69ms/step - loss: 0.6917 - accuracy: 0.7133 - val_loss: 0.8537 - val_accuracy: 0.6588
Epoch 4/5
976/976 [==============================] - 67s 69ms/step - loss: 0.6724 - accuracy: 0.7196 - val_loss: 0.8565 - val_accuracy: 0.6605
Epoch 5/5
976/976 [==============================] - 66s 68ms/step - loss: 0.6525 - accuracy: 0.7279 - val_loss: 0.8759 - val_accuracy: 0.6591
Wall time: 5min 34s


In [113]:
y_pred1=model1.predict_classes(X_test,verbose=1)


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
2072/2072 [==============================] - 10s 5ms/step


In [133]:
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding


In [134]:

model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.2))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(num_classes,activation='softmax'))


model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 29, 100)           1360400   
_________________________________________________________________
dropout_3 (Dropout)          (None, 29, 100)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 29, 64)            19264     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                

In [136]:
%%time

history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/5
976/976 [==============================] - 18s 19ms/step - loss: 0.6020 - accuracy: 0.7433 - val_loss: 0.8857 - val_accuracy: 0.6605
Epoch 2/5
976/976 [==============================] - 18s 19ms/step - loss: 0.5787 - accuracy: 0.7504 - val_loss: 0.9089 - val_accuracy: 0.6450
Epoch 3/5
976/976 [==============================] - 19s 19ms/step - loss: 0.5593 - accuracy: 0.7576 - val_loss: 0.9327 - val_accuracy: 0.6554
Epoch 4/5
976/976 [==============================] - 19s 19ms/step - loss: 0.5418 - accuracy: 0.7623 - val_loss: 0.9613 - val_accuracy: 0.6525
Epoch 5/5
976/976 [==============================] - 19s 19ms/step - loss: 0.5269 - accuracy: 0.7669 - val_loss: 1.0362 - val_accuracy: 0.6516
Wall time: 1min 33s
